In [1]:
%load_ext autoreload
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
import torch

from mvn.utils.minimon import MiniMon
from mvn.utils.misc import flush_cache

from mvn.ipynb import get_args, get_config, build_labels
from train import setup_dataloaders, setup_experiment, build_env, do_train


flush_cache()

|===========================================================================|
|                  PyTorch CUDA memory summary, device ID 0                 |
|---------------------------------------------------------------------------|
|            CUDA OOMs: 0            |        cudaMalloc retries: 0         |
|===========================================================================|
|        Metric         | Cur Usage  | Peak Usage | Tot Alloc  | Tot Freed  |
|---------------------------------------------------------------------------|
| Allocated memory      |       0 B  |       0 B  |       0 B  |       0 B  |
|       from large pool |       0 B  |       0 B  |       0 B  |       0 B  |
|       from small pool |       0 B  |       0 B  |       0 B  |       0 B  |
|---------------------------------------------------------------------------|
| Active memory         |       0 B  |       0 B  |       0 B  |       0 B  |
|       from large pool |       0 B  |       0 B  |       0 B  |

In [3]:
is_distributed = False
master = True
device = torch.device(0)
print('using dev {}'.format(device))
    
args = get_args()

# just to get a feeling of the dataset
# labels, mask, indices = build_labels(config.dataset.train.labels_path, 10000)
# labels, mask, indices = build_labels(config.dataset.train.labels_path, 10, allowed_subjects=['S9', 'S11'])

using dev cuda:0
# available GPUs: 1


In [58]:
config = get_config(args)
do_train(None, None, config, device, is_distributed, master)

                         conv1 has       9408 params (~  0.0) %
                           bn1 has        128 params (~  0.0) %
                          relu has          0 params (~  0.0) %
                       maxpool has          0 params (~  0.0) %
                        layer1 has     147968 params (~  0.8) %
                        layer2 has     525568 params (~  2.7) %
                        layer3 has    2099712 params (~ 10.9) %
                        layer4 has    8393728 params (~ 43.8) %
               alg_confidences has    3808529 params (~ 19.9) %
                 deconv_layers has    4195840 params (~ 21.9) %
                   final_layer has       4369 params (~  0.0) %
total params: 19185250
freezing Conv2d
freezing BatchNorm2d
freezing ReLU
freezing MaxPool2d
freezing Sequential
freezing Sequential
freezing Sequential
freezing Sequential
                         conv1 has          0 params (~  0.0) %
                           bn1 has          0 params (~  0.

Exception ignored in: <function _MultiProcessingDataLoaderIter.__del__ at 0x7f4df8b223a0>
Traceback (most recent call last):
  File "/home/stefano/scratch/learnable-triangulation-pytorch/.venv/lib/python3.8/site-packages/torch/utils/data/dataloader.py", line 1324, in __del__
    self._shutdown_workers()
  File "/home/stefano/scratch/learnable-triangulation-pytorch/.venv/lib/python3.8/site-packages/torch/utils/data/dataloader.py", line 1316, in _shutdown_workers
    if w.is_alive():
  File "/usr/lib/python3.8/multiprocessing/process.py", line 160, in is_alive
    assert self._parent_pid == os.getpid(), 'can only test a child process'
AssertionError: can only test a child process


#[cam2cam]: training batch iter 0 loss ~ 19895.176
#[cam2cam]: training batch iter 1 loss ~ 10671.017
#[epoch]: training MPJPE relative to pelvis: 12007.562 mm
#[epoch]: eval MPJPE relative to pelvis: 534.545 mm
epoch    1 has started!
#[cam2cam]: training batch iter 0 loss ~ 19153.672
#[cam2cam]: training batch iter 1 loss ~ 8701.301
#[epoch]: training MPJPE relative to pelvis: 312811.438 mm
#[epoch]: eval MPJPE relative to pelvis: 471.507 mm
    evaluate results x          4 ~        0.0 [min:        0.0,    max:        0.0,    last:        0.0]
       alg: backbone x          6 ~        0.0 [min:        0.0,    max:        0.0,    last:        0.0]
     cam2cam forward x          6 ~        0.0 [min:        0.0,    max:        0.0,    last:        0.0]
           calc loss x          4 ~        0.0 [min:        0.0,    max:        0.0,    last:        0.0]
     BB forward pass x          6 ~        0.0 [min:        0.0,    max:        0.0,    last:        0.0]
         cam2cam DLT x

In [53]:
BN_MOMENTUM = 0.1
batch_size = 3
heatmaps_w, heatmaps_h = 32, 32
n_joints = 17

batch = torch.zeros(batch_size, 2, n_joints, heatmaps_w, heatmaps_h)
print('{:>20}: {:>80}'.format('original', str(batch.shape)))

batch = batch.view(batch_size * 2, n_joints, heatmaps_w, heatmaps_h)  # process each view with same weights
batch = torch.nn.Conv2d(17, 17, 3, stride=1, padding=0, bias=True, padding_mode='zeros')(batch)
print('{:>20}: {:>80}'.format('after CN', str(batch.shape)))

batch = torch.nn.LeakyReLU(inplace=True)(batch)
print('{:>20}: {:>80}'.format('after ReLU', str(batch.shape)))

batch = torch.nn.MaxPool2d(kernel_size=3, stride=2, padding=1)(batch)
print('{:>20}: {:>80}'.format('after MaxPool', str(batch.shape)))

            original:                                                   torch.Size([3, 2, 17, 32, 32])
            after CN:                                                      torch.Size([6, 17, 30, 30])
          after ReLU:                                                      torch.Size([6, 17, 30, 30])
       after MaxPool:                                                      torch.Size([6, 17, 15, 15])


In [39]:
batch_size = 3
n_joints = 17

batch = torch.zeros(batch_size, 2, n_joints, 2)
print('{:>20}: {:>80}'.format('original', str(batch.shape)))

batch = torch.nn.BatchNorm2d(num_features=2, momentum=BN_MOMENTUM)(batch)
print('{:>20}: {:>80}'.format('after bn', str(batch.shape)))

batch = batch.view(batch_size, 2 * n_joints * 2)
print('{:>20}: {:>80}'.format('after view', str(batch.shape)))

            original:                                                        torch.Size([3, 2, 17, 2])
            after bn:                                                        torch.Size([3, 2, 17, 2])
          after view:                                                              torch.Size([3, 68])
